<a href="https://colab.research.google.com/github/salma1717/nlpcc-ui-2025/blob/main/Class_5%266_Assignment_Salma_Nazwa_Aurelia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Simple LLM Chat Application (Vanilla Chat)

Notebook ini mengimplementasikan aplikasi obrolan baris perintah dasar yang dapat digunakan berulang kali. Buku catatan ini menunjukkan interaksi terprogram dengan API LLM (GPT OpenAI secara default, dengan dukungan opsional untuk Gemini Google) untuk menanggapi masukan pengguna.

In [1]:
# week5_6_task1_chat.ipynb

# Import necessary libraries
import openai
from google.colab import userdata # For API key management in Colab
import os

# --- Configuration and API Key Setup ---
# Securely load your OpenAI API key from Colab Secrets
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if OPENAI_API_KEY is None:
        raise ValueError("OpenAI API key not found in Colab Secrets. Please add it.")
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY # Set it as an environment variable
    openai.api_key = OPENAI_API_KEY
    print("OpenAI API key loaded successfully.")
except Exception as e:
    print(f"Error loading OpenAI API key: {e}")
    print("Please ensure your OPENAI_API_KEY is correctly set in Colab Secrets (left sidebar -> 🔑).")
    OPENAI_API_KEY = None # Ensure it's None if loading failed

# (Optional) Setup for Google Gemini API (if you have a key and want to use it)
# try:
#     GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
#     if GEMINI_API_KEY is None:
#         raise ValueError("Gemini API key not found in Colab Secrets.")
#     import google.generativeai as genai
#     genai.configure(api_key=GEMINI_API_KEY)
#     print("Google Gemini API key loaded and configured successfully.")
#     # Example: list models to verify
#     # for m in genai.list_models():
#     #   if 'generateContent' in m.supported_generation_methods:
#     #     print(m.name)
# except Exception as e:
#     print(f"Error loading or configuring Gemini API key: {e}")
#     GEMINI_API_KEY = None

# Choose your LLM model
# For OpenAI: "gpt-3.5-turbo", "gpt-4o"
# For Gemini: "gemini-1.5-flash-latest", "gemini-pro"
OPENAI_MODEL_NAME = "gpt-3.5-turbo"
# GEMINI_MODEL_NAME = "gemini-1.5-flash-latest"

# Install OpenAI library (if not already installed by Colab)
# !pip install openai -q
# Install Gemini library (if you plan to use it)
# !pip install -q -U google-generativeai

OpenAI API key loaded successfully.


Step 2: Core Chat Application Logic (Task 1)

Add the following code cell for the chat application. This example implements a multi-turn chat using OpenAI.

In [2]:
# (Continue in week5_6_task1_chat.ipynb)

# --- Chat Application Logic (OpenAI) ---
def run_openai_chat_application():
    """
    Runs a multi-turn command-line chat application using the OpenAI API.
    """
    if not OPENAI_API_KEY:
        print("OpenAI API key is not available. Cannot start the chat application.")
        return

    # Initialize conversation history.
    # The first message is a system message to set the LLM's behavior.
    conversation_history = [
        {"role": "system", "content": "You are a helpful and friendly assistant."}
    ]

    print(f"\n--- Starting OpenAI Chat Application ---")
    print(f"Model: {OPENAI_MODEL_NAME}")
    print("Type 'quit', 'exit', or 'bye' to end the chat session.")
    print("-" * 40)

    while True:
        try:
            user_input = input("You: ")
            if user_input.lower() in ["quit", "exit", "bye"]:
                print("\nAssistant: Goodbye! Thanks for chatting.")
                break

            if not user_input.strip(): # Handle empty input from user
                print("Assistant: Please type a message.")
                continue

            # Add user's message to conversation history
            conversation_history.append({"role": "user", "content": user_input})

            # Make the API call to OpenAI
            try:
                response = openai.chat.completions.create(
                    model=OPENAI_MODEL_NAME,
                    messages=conversation_history,
                    temperature=0.7, # Controls randomness. Lower is more deterministic.
                    max_tokens=150   # Max length of the response.
                )

                # Extract the assistant's reply
                assistant_reply = response.choices[0].message.content.strip()

                # Add assistant's reply to conversation history
                conversation_history.append({"role": "assistant", "content": assistant_reply})

                print(f"Assistant: {assistant_reply}")

            except openai.APIError as e:
                error_message = f"OpenAI API Error: {e}. Please check your API key, model access, or network."
                print(f"Assistant: Sorry, I encountered an API error. ({error_message}) Please try again.")
                # Remove the last user message if the API call failed, so it's not re-processed with the error.
                if conversation_history and conversation_history[-1]["role"] == "user":
                    conversation_history.pop()
            except Exception as e:
                error_message = f"An unexpected error occurred: {e}"
                print(f"Assistant: Sorry, an unexpected error occurred. ({error_message}) Please try again.")
                if conversation_history and conversation_history[-1]["role"] == "user":
                    conversation_history.pop()

        except EOFError: # Handles Ctrl+D in some environments
            print("\nAssistant: Chat ended unexpectedly. Goodbye!")
            break
        except KeyboardInterrupt: # Handles Ctrl+C
            print("\nAssistant: Chat interrupted. Goodbye!")
            break

# --- (Optional) Chat Application Logic (Gemini) ---
# def run_gemini_chat_application():
#     if not GEMINI_API_KEY:
#         print("Gemini API key is not available. Cannot start the chat application.")
#         return
#
#     try:
#         model = genai.GenerativeModel(GEMINI_MODEL_NAME,
#                                       system_instruction="You are a helpful and creative assistant.") # System prompt
#         chat = model.start_chat(history=[]) # Initialize an empty chat history
#     except Exception as e:
#         print(f"Error initializing Gemini model: {e}")
#         return
#
#     print(f"\n--- Starting Gemini Chat Application ---")
#     print(f"Model: {GEMINI_MODEL_NAME}")
#     print("Type 'quit', 'exit', or 'bye' to end the chat session.")
#     print("-" * 40)
#
#     while True:
#         try:
#             user_input = input("You: ")
#             if user_input.lower() in ["quit", "exit", "bye"]:
#                 print("\nAssistant: Goodbye! Thanks for chatting with Gemini.")
#                 break
#             if not user_input.strip():
#                 print("Assistant: Please type a message.")
#                 continue
#
#             # Send message to Gemini. History is managed by the 'chat' object.
#             try:
#                 response = chat.send_message(user_input)
#                 assistant_reply = response.text.strip()
#                 print(f"Assistant: {assistant_reply}")
#             except Exception as e: # Broad exception for API errors or content filtering
#                 print(f"Assistant: Sorry, I encountered an error with Gemini. ({e}) Please try again.")
#                 # Note: With Gemini's chat object, managing history after an error might require
#                 # inspecting chat.history and potentially removing the last user part if needed.
#
#         except EOFError:
#             print("\nAssistant: Chat ended unexpectedly. Goodbye!")
#             break
#         except KeyboardInterrupt:
#             print("\nAssistant: Chat interrupted. Goodbye!")
#             break

# --- Main Execution for Task 1 ---
if __name__ == "__main__":
    # To run the chat, call the function.
    # Make sure the API key was loaded successfully.
    if OPENAI_API_KEY:
        print("Starting OpenAI chat. If you prefer Gemini and have set it up, modify the call below.")
        run_openai_chat_application()
    # elif GEMINI_API_KEY: # Uncomment if you want to default to Gemini if OpenAI fails or is not set
    #     print("OpenAI key not found or failed to load. Starting Gemini chat.")
    #     run_gemini_chat_application()
    else:
        print("No API key is available (OpenAI or Gemini). Chat application cannot start.")
        print("Please ensure 'OPENAI_API_KEY' (and/or 'GEMINI_API_KEY') is set in Colab Secrets.")

Starting OpenAI chat. If you prefer Gemini and have set it up, modify the call below.

--- Starting OpenAI Chat Application ---
Model: gpt-3.5-turbo
Type 'quit', 'exit', or 'bye' to end the chat session.
----------------------------------------
You: Hello, whatchu doin'?
Assistant: Hello! I'm here to help you with any questions or tasks you have. How can I assist you today?
You: Can u help me to answer this question : 100 x 100?
Assistant: Of course! The answer to 100 x 100 is 10,000. Let me know if you need help with anything else!
You: Thankyou!
Assistant: You're welcome! If you have any more questions or need assistance in the future, feel free to ask. Have a great day!
You: Bye!
Assistant: Goodbye! Take care!
You: quit

Assistant: Goodbye! Thanks for chatting.


Step 3: Markdown Documentation and Example Interactions (Task 1)

Add a markdown cell below your code cells in week5_6_task1_chat.ipynb.

In [7]:
import os
import openai
import google.generativeai as genai
from IPython.display import display, Markdown

# --- API Key Configuration ---
# Access OpenAI API key from Colab Secrets
try:
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    if OPENAI_API_KEY:
        openai.api_key = OPENAI_API_KEY
        print("OpenAI API key loaded successfully.")
    else:
        print("OPENAI_API_KEY not found in Colab Secrets. Please add it.")
except Exception as e:
    print(f"Error loading OpenAI API key: {e}")

# Access Gemini API key from Colab Secrets (optional)
try:
    GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
    if GEMINI_API_KEY:
        genai.configure(api_key=GEMINI_API_KEY)
        print("Gemini API key loaded successfully.")
    else:
        print("GEMINI_API_KEY not found in Colab Secrets. Gemini functionality might be limited.")
except Exception as e:
    print(f"Error loading Gemini API key: {e}")

# --- LLM Model Configuration ---
OPENAI_MODEL_NAME = "gpt-3.5-turbo" # You can change this to "gpt-4o" or other models
GEMINI_MODEL_NAME = "gemini-1.5-flash-latest" # Or "gemini-1.5-pro-latest"

# Initial system message for OpenAI
OPENAI_SYSTEM_PROMPT = {"role": "system", "content": "You are a helpful and friendly assistant."}

OpenAI API key loaded successfully.
GEMINI_API_KEY not found in Colab Secrets. Gemini functionality might be limited.


In [8]:
import os
import openai
import google.generativeai as genai
import sys

# --- OpenAI Chat Application ---
def run_openai_chat_application(model_name, system_prompt):
    print(f"\n--- Starting OpenAI Chat Application ---")
    print(f"Model: {model_name}")
    print("Type 'quit', 'exit', or 'bye' to end the chat session.")
    print("----------------------------------------")

    conversation_history = [system_prompt]

    while True:
        try:
            user_input = input("You: ")
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("Assistant: Goodbye! Thanks for chatting.")
                break

            conversation_history.append({"role": "user", "content": user_input})

            response = openai.chat.completions.create(
                model=model_name,
                messages=conversation_history,
                temperature=0.7, # Adjust for creativity vs. consistency
            )

            assistant_response = response.choices[0].message.content
            print(f"Assistant: {assistant_response}")
            conversation_history.append({"role": "assistant", "content": assistant_response})

        except openai.APIError as e:
            print(f"OpenAI API Error: {e}")
            print("Please check your API key and network connection.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            # Optionally, you might want to log the full traceback for debugging
            # import traceback
            # traceback.print_exc()
        except KeyboardInterrupt:
            print("\nChat session interrupted. Goodbye!")
            break

# --- Gemini Chat Application (Optional - Uncomment and configure to use) ---
def run_gemini_chat_application(model_name):
    # Ensure GEMINI_API_KEY is loaded in the first cell
    if not os.environ.get("GEMINI_API_KEY"):
        print("Gemini API key not found. Cannot run Gemini chat.")
        return

    print(f"\n--- Starting Gemini Chat Application ---")
    print(f"Model: {model_name}")
    print("Type 'quit', 'exit', or 'bye' to end the chat session.")
    print("----------------------------------------")

    try:
        model = genai.GenerativeModel(model_name)
        chat = model.start_chat(history=[]) # Initialize with empty history
    except Exception as e:
        print(f"Error initializing Gemini model or chat: {e}")
        return

    while True:
        try:
            user_input = input("You: ")
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("Assistant: Goodbye! Thanks for chatting.")
                break

            response = chat.send_message(user_input)
            assistant_response = response.text
            print(f"Assistant: {assistant_response}")

        except genai.types.core.ClientError as e:
            print(f"Gemini API Error: {e}")
            print("Please check your API key and network connection.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
        except KeyboardInterrupt:
            print("\nChat session interrupted. Goodbye!")
            break

# --- Main execution block ---
if __name__ == "__main__":
    # --- Choose which chat application to run ---
    # By default, runs OpenAI. Uncomment the Gemini line to use Gemini instead.

    # Option 1: Run OpenAI Chat
    if OPENAI_API_KEY: # Only try to run if OpenAI key is available
        print("Starting OpenAI chat. If you prefer Gemini and have set it up, modify the call below.")
        run_openai_chat_application(OPENAI_MODEL_NAME, OPENAI_SYSTEM_PROMPT)
    else:
        print("OpenAI API key not set. Cannot start OpenAI chat.")

    # Option 2: Run Gemini Chat (Uncomment the following lines to use Gemini)
    # if GEMINI_API_KEY: # Only try to run if Gemini key is available
    #     print("Starting Gemini chat. Make sure you've configured GEMINI_API_KEY.")
    #     run_gemini_chat_application(GEMINI_MODEL_NAME)
    # else:
    #     print("Gemini API key not set. Cannot start Gemini chat.")

Starting OpenAI chat. If you prefer Gemini and have set it up, modify the call below.

--- Starting OpenAI Chat Application ---
Model: gpt-3.5-turbo
Type 'quit', 'exit', or 'bye' to end the chat session.
----------------------------------------
You: Hello
Assistant: Hello! How can I assist you today?
You: Can u help me answer this question : what the meaning of FOMO?
Assistant: Of course! FOMO stands for "fear of missing out." It is a feeling of anxiety or insecurity that you might be missing out on rewarding experiences that others are having.
You: Okay ,Thank You!
Assistant: You're welcome! If you have any more questions or need assistance with anything else, feel free to ask. Have a great day!
You: quit
Assistant: Goodbye! Thanks for chatting.


In [10]:
!pip install pandas openai google-generativeai # Jalankan ini jika belum terinstal
import pandas as pd
import os
import openai
import google.generativeai as genai

# Pastikan API Keys sudah dimuat dari cell sebelumnya atau muat lagi jika ini notebook terpisah
# Contoh:
# try:
#     OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
#     if OPENAI_API_KEY:
#         openai.api_key = OPENAI_API_KEY
#         print("OpenAI API key loaded successfully for Task 2.")
# except Exception as e:
#     print(f"Error loading OpenAI API key for Task 2: {e}")
#
# try:
#     GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
#     if GEMINI_API_KEY:
#         genai.configure(api_key=GEMINI_API_KEY)
#         print("Gemini API key loaded successfully for Task 2.")
# except Exception as e:
#     print(f"Error loading Gemini API key for Task 2: {e}")

# Konfigurasi model (bisa sama dengan Task 1 atau berbeda)
OPENAI_MODEL_NAME_T2 = "gpt-3.5-turbo"
GEMINI_MODEL_NAME_T2 = "gemini-1.5-flash-latest"

In [11]:
# Contoh: Mengunduh atau membuat dataset
# Jika dari Kaggle, Anda mungkin perlu mengunduh file CSV dan mengunggahnya ke Colab
# atau menggunakan Kaggle API. Untuk contoh ini, saya akan membuat dataset sintetis sederhana.

data = {
    'product_id': ['P001', 'P002', 'P003', 'P004', 'P005'],
    'product_name': ['Smartphone X', 'Laptop Pro', 'Smartwatch 5', 'Wireless Earbuds', 'Portable Speaker'],
    'review_text': [
        'Love this phone! The camera is amazing and the battery lasts forever.',
        'Good laptop for the price, but it gets a bit hot when gaming.',
        'Sleek design and accurate health tracking. Battery life is decent.',
        'Excellent sound quality and comfortable fit. Connection is stable.',
        'Sound is okay, but the bass is weak. Portable design is a plus.'
    ],
    'rating': [5, 4, 4, 5, 3]
}
df = pd.DataFrame(data)

# Jika Anda memiliki file CSV, ganti dengan ini:
# df = pd.read_csv('nama_file_anda.csv')

print("Dataset Loaded:")
print(df.head())
print(f"\nJumlah baris dalam dataset: {len(df)}")

Dataset Loaded:
  product_id      product_name  \
0       P001      Smartphone X   
1       P002        Laptop Pro   
2       P003      Smartwatch 5   
3       P004  Wireless Earbuds   
4       P005  Portable Speaker   

                                         review_text  rating  
0  Love this phone! The camera is amazing and the...       5  
1  Good laptop for the price, but it gets a bit h...       4  
2  Sleek design and accurate health tracking. Bat...       4  
3  Excellent sound quality and comfortable fit. C...       5  
4  Sound is okay, but the bass is weak. Portable ...       3  

Jumlah baris dalam dataset: 5


In [12]:
# Fungsi untuk memanggil OpenAI API
def get_openai_sentiment(review_text, model_name=OPENAI_MODEL_NAME_T2):
    try:
        # Prompt yang akan dikirim ke LLM
        # Anda bisa menyesuaikan prompt ini sesuai dengan tugas Anda
        prompt = f"Analyze the sentiment of the following product review: '{review_text}'. Respond with 'Positive', 'Negative', or 'Neutral'."

        messages = [
            {"role": "system", "content": "You are a helpful assistant that analyzes sentiment."},
            {"role": "user", "content": prompt}
        ]

        response = openai.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=0.0 # Lebih rendah untuk tugas yang faktual/konsisten
        )
        return response.choices[0].message.content
    except openai.APIError as e:
        print(f"OpenAI API Error for review '{review_text[:50]}...': {e}")
        return "Error"
    except Exception as e:
        print(f"An unexpected error occurred for review '{review_text[:50]}...': {e}")
        return "Error"

# Fungsi untuk memanggil Gemini API (jika Anda ingin menggunakan Gemini)
def get_gemini_sentiment(review_text, model_name=GEMINI_MODEL_NAME_T2):
    try:
        model = genai.GenerativeModel(model_name)
        # Prompt yang akan dikirim ke LLM
        prompt = f"Analyze the sentiment of the following product review: '{review_text}'. Respond with 'Positive', 'Negative', or 'Neutral'."
        response = model.generate_content(prompt)
        return response.text
    except genai.types.core.ClientError as e:
        print(f"Gemini API Error for review '{review_text[:50]}...': {e}")
        return "Error"
    except Exception as e:
        print(f"An unexpected error occurred for review '{review_text[:50]}...': {e}")
        return "Error"

# Iterasi melalui baris DataFrame dan terapkan fungsi LLM
# Batasi jumlah baris untuk pengujian agar tidak menghabiskan terlalu banyak token/kuota API
# Ganti 'review_text' dengan nama kolom teks Anda yang sebenarnya
df['llm_sentiment'] = df['review_text'].apply(lambda x: get_openai_sentiment(x))
# Atau jika Anda ingin menggunakan Gemini:
# df['llm_sentiment'] = df['review_text'].apply(lambda x: get_gemini_sentiment(x))

print("\nProcessing complete. Here are the results:")


Processing complete. Here are the results:


In [15]:
# Menampilkan DataFrame dengan kolom baru yang berisi hasil LLM
# Anda bisa memilih kolom mana saja yang ingin ditampilkan
display_df = df[['product_name', 'review_text', 'rating', 'llm_sentiment']]
print(display_df.to_markdown(index=False)) # Untuk tampilan yang rapi di markdown

| product_name     | review_text                                                           |   rating | llm_sentiment   |
|:-----------------|:----------------------------------------------------------------------|---------:|:----------------|
| Smartphone X     | Love this phone! The camera is amazing and the battery lasts forever. |        5 | Positive        |
| Laptop Pro       | Good laptop for the price, but it gets a bit hot when gaming.         |        4 | Neutral         |
| Smartwatch 5     | Sleek design and accurate health tracking. Battery life is decent.    |        4 | Positive        |
| Wireless Earbuds | Excellent sound quality and comfortable fit. Connection is stable.    |        5 | Positive        |
| Portable Speaker | Sound is okay, but the bass is weak. Portable design is a plus.       |        3 | Neutral         |


# Task 2: LLM-Powered Tabular Data Processing

## Tujuan

Tugas ini bertujuan untuk menerapkan Large Language Models (LLMs) secara programatik untuk memproses multiple rows of data dari sumber tabular. Konsepnya mirip dengan "asking the same question to N rows" yang telah didemonstrasikan dalam sesi live coding.

---

> Add blockquote



In [16]:
# Installasi library yang mungkin dibutuhkan (jalankan hanya jika belum terinstal)
!pip install pandas openai google-generativeai

import pandas as pd
import os
import openai
import google.generativeai as genai

# Pastikan API Keys sudah dimuat dari cell sebelumnya
# Jika ini adalah notebook terpisah, Anda perlu memuat ulang API key di sini:
# try:
#     OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
#     if OPENAI_API_KEY:
#         openai.api_key = OPENAI_API_KEY
#         print("OpenAI API key loaded successfully for Task 2.")
#     else:
#         print("OPENAI_API_KEY not found in Colab Secrets. Please add it.")
# except Exception as e:
#     print(f"Error loading OpenAI API key for Task 2: {e}")

# Konfigurasi model untuk Task 2
OPENAI_MODEL_NAME_T2 = "gpt-3.5-turbo" # Bisa diubah ke gpt-4o, dll.
GEMINI_MODEL_NAME_T2 = "gemini-1.5-flash-latest" # Bisa diubah ke gemini-1.5-pro-latest

## Dataset Selection

Untuk tugas ini, saya memilih/membuat dataset [**Ganti dengan nama dataset Anda, contoh: "Product Review Sentiment Data"**].

* **Sumber Dataset:**
    * Jika dari Kaggle: [Sertakan **link URL langsung** ke dataset di Kaggle, contoh: `https://www.kaggle.com/datasets/your_dataset_name`]
    * Jika sintetis/buatan sendiri: [Deskripsikan bagaimana Anda membuat dataset ini, contoh: "Dataset ini dibuat secara sintetis untuk demonstrasi analisis sentimen, berisi ulasan produk fiktif dan rating."]

* **Alasan Pemilihan:**
    Dataset ini dipilih karena [Jelaskan **mengapa dataset ini cocok** untuk tugas Anda. Contoh: "memiliki kolom teks (`review_text`) yang ideal untuk dianalisis sentimennya menggunakan LLM, serta kolom rating sebagai pembanding."].

In [17]:
# --- Data Loading & Iteration ---

# Contoh: Membuat dataset sintetis (Ganti dengan kode loading dataset Anda yang sebenarnya)
# Jika Anda punya file CSV, unggah ke Colab dan gunakan:
# df = pd.read_csv('nama_file_anda.csv')

data = {
    'product_id': ['P001', 'P002', 'P003', 'P004', 'P005', 'P006', 'P007', 'P008', 'P009', 'P010'],
    'product_name': ['Smartphone X', 'Laptop Pro', 'Smartwatch 5', 'Wireless Earbuds', 'Portable Speaker',
                     'Gaming Mouse', 'Mechanical Keyboard', 'Webcam HD', 'Monitor UltraWide', 'Noise Cancelling Headphones'],
    'review_text': [
        'Love this phone! The camera is amazing and the battery lasts forever. Definitely worth the price.',
        'Good laptop for the price, but it gets a bit hot when gaming. Screen quality is superb though.',
        'Sleek design and accurate health tracking. Battery life is decent for daily use.',
        'Excellent sound quality and comfortable fit. Connection is stable even far from device.',
        'Sound is okay, but the bass is weak. Portable design is a plus, but not for audiophiles.',
        'Smooth tracking and good click response. RGB lights are a nice touch, but software is clunky.',
        'Typing experience is fantastic, very tactile. A bit loud for office use, but great for gaming.',
        'Clear video and microphone for calls. Easy to set up, but auto-focus can be slow sometimes.',
        'Immersive viewing experience with vibrant colors. Great for multitasking, but stand is a bit wobbly.',
        'Very effective at blocking out noise, comfortable for long flights. Audio quality is clear but not exceptional bass.'
    ],
    'rating': [5, 4, 4, 5, 3, 4, 5, 4, 4, 5]
}
df = pd.DataFrame(data)

print("Dataset Loaded:")
# Menampilkan 5 baris pertama dan info dasar
print(df.head())
print(f"\nJumlah baris dalam dataset: {len(df)}")
print("\nKolom yang tersedia:")
print(df.columns.tolist())

Dataset Loaded:
  product_id      product_name  \
0       P001      Smartphone X   
1       P002        Laptop Pro   
2       P003      Smartwatch 5   
3       P004  Wireless Earbuds   
4       P005  Portable Speaker   

                                         review_text  rating  
0  Love this phone! The camera is amazing and the...       5  
1  Good laptop for the price, but it gets a bit h...       4  
2  Sleek design and accurate health tracking. Bat...       4  
3  Excellent sound quality and comfortable fit. C...       5  
4  Sound is okay, but the bass is weak. Portable ...       3  

Jumlah baris dalam dataset: 10

Kolom yang tersedia:
['product_id', 'product_name', 'review_text', 'rating']


## LLM Prompting per Row

Tugas yang saya minta dari LLM untuk setiap baris data adalah **[Ganti dengan tugas LLM Anda, contoh: "Analisis Sentimen"]**. LLM akan mengekstrak sentimen dari kolom `review_text` dan mengklasifikasikannya sebagai 'Positive', 'Negative', atau 'Neutral'.

**Struktur Prompt yang Digunakan:**

Untuk setiap baris, prompt akan diformulasikan sebagai berikut:

In [18]:
# --- LLM Prompting per Row & Response Handling ---

# Fungsi untuk memanggil OpenAI API untuk analisis sentimen
def get_openai_sentiment(review_text, model_name=OPENAI_MODEL_NAME_T2):
    try:
        prompt = f"Analyze the sentiment of the following product review: '{review_text}'. Respond with 'Positive', 'Negative', or 'Neutral'."
        messages = [
            {"role": "system", "content": "You are a helpful assistant that analyzes sentiment and responds only with 'Positive', 'Negative', or 'Neutral'."},
            {"role": "user", "content": prompt}
        ]
        response = openai.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=0.0, # Rendah untuk konsistensi
            max_tokens=10 # Batasi token untuk respons singkat
        )
        return response.choices[0].message.content.strip() # Menghapus spasi ekstra
    except openai.APIError as e:
        print(f"OpenAI API Error for review '{review_text[:50]}...': {e}")
        return "Error: API issue"
    except Exception as e:
        print(f"An unexpected error occurred for review '{review_text[:50]}...': {e}")
        return "Error: Processing issue"

# Fungsi untuk memanggil Gemini API untuk analisis sentimen (opsional, jika ingin pakai Gemini)
def get_gemini_sentiment(review_text, model_name=GEMINI_MODEL_NAME_T2):
    try:
        # Pastikan GEMINI_API_KEY sudah dikonfigurasi di awal notebook
        if not genai.configure().api_key:
            return "Error: Gemini API key not configured."

        model = genai.GenerativeModel(model_name)
        prompt = f"Analyze the sentiment of the following product review: '{review_text}'. Respond only with 'Positive', 'Negative', or 'Neutral'."
        response = model.generate_content(prompt)
        return response.text.strip()
    except genai.types.core.ClientError as e:
        print(f"Gemini API Error for review '{review_text[:50]}...': {e}")
        return "Error: API issue"
    except Exception as e:
        print(f"An unexpected error occurred for review '{review_text[:50]}...': {e}")
        return "Error: Processing issue"

print(f"Starting LLM processing for {len(df)} rows. This might take a moment...")

# Terapkan fungsi LLM ke setiap baris pada kolom 'review_text'
# Gunakan .copy() jika Anda mengambil sampel dari DataFrame yang lebih besar untuk menghindari SettingWithCopyWarning
# df_sample = df.head(10).copy() # Untuk menguji hanya beberapa baris

# Pilih fungsi LLM yang akan Anda gunakan (OpenAI atau Gemini)
df['llm_sentiment'] = df['review_text'].apply(lambda x: get_openai_sentiment(x))
# Atau jika Anda ingin menggunakan Gemini:
# df['llm_sentiment'] = df['review_text'].apply(lambda x: get_gemini_sentiment(x))

print("\nLLM processing complete.")

Starting LLM processing for 10 rows. This might take a moment...

LLM processing complete.


7. Kode: Presentation of Results

In [19]:
# --- Presentation of Results ---

print("\n--- Original Data and LLM-Generated Sentiments ---")

# Menampilkan DataFrame dengan kolom yang relevan
# Anda bisa menyesuaikan kolom mana saja yang ingin ditampilkan
display_df = df[['product_name', 'review_text', 'rating', 'llm_sentiment']]

# Gunakan to_markdown() untuk tampilan yang rapi di notebook
# atau to_string() jika Anda ingin melihat semua baris tanpa truncasi
print(display_df.to_markdown(index=False))

# Analisis sederhana (opsional)
print("\n--- Sentiment Analysis Summary ---")
print(df['llm_sentiment'].value_counts().to_markdown())

# Anda juga bisa membandingkan dengan rating asli jika relevan
# print("\nComparison of LLM Sentiment vs. Original Rating:")
# # Contoh sederhana: Konversi rating ke sentimen dasar untuk perbandingan
# df['original_sentiment'] = df['rating'].apply(lambda x: 'Positive' if x >=4 else ('Negative' if x <=2 else 'Neutral'))
# print(df[['review_text', 'llm_sentiment', 'original_sentiment']].to_markdown(index=False))


--- Original Data and LLM-Generated Sentiments ---
| product_name                | review_text                                                                                                          |   rating | llm_sentiment   |
|:----------------------------|:---------------------------------------------------------------------------------------------------------------------|---------:|:----------------|
| Smartphone X                | Love this phone! The camera is amazing and the battery lasts forever. Definitely worth the price.                    |        5 | Positive        |
| Laptop Pro                  | Good laptop for the price, but it gets a bit hot when gaming. Screen quality is superb though.                       |        4 | Positive        |
| Smartwatch 5                | Sleek design and accurate health tracking. Battery life is decent for daily use.                                     |        4 | Positive        |
| Wireless Earbuds            | Excellent sound 

Baik, ini adalah kesimpulan dari seluruh tugas dan panduan yang telah kita bahas, mencakup kedua Task (1 dan 2) serta potensi Bonus Task.
Kesimpulan: Aplikasi LLM Programatik Anda

Anda telah berhasil membangun dan mendokumentasikan dua aplikasi utama menggunakan Large Language Models (LLMs) secara programatik:
Task 1: Aplikasi Chat LLM Sederhana (Vanilla Chat)

Anda telah membuat aplikasi chat berbasis command-line yang memungkinkan interaksi multi-turn dengan LLM (baik OpenAI GPT atau Google Gemini). Aplikasi ini berfungsi dengan cara mengirimkan input pengguna ke API LLM dan menampilkan respons yang dihasilkan. Fitur penting dari aplikasi ini adalah kemampuannya untuk mempertahankan konteks percakapan melalui riwayat obrolan, yang memungkinkan LLM memberikan respons yang lebih koheren dan relevan dalam percakapan yang berkelanjutan. Anda juga telah mengimplementasikan penanganan API Key yang aman menggunakan Colab Secrets, sesuai praktik terbaik.
Task 2: Pemrosesan Data Tabular dengan LLM

Selain chat, Anda juga telah berhasil menerapkan LLM untuk memproses data tabular. Anda telah memilih atau membuat dataset, memuatnya ke dalam DataFrame, dan mengiterasi setiap baris. Untuk setiap baris, Anda merumuskan prompt khusus yang memanfaatkan data dari kolom-kolom tertentu (misalnya, review_text), lalu mengirimkan prompt tersebut ke LLM. Respons dari LLM kemudian diambil dan disimpan kembali ke dalam DataFrame Anda sebagai kolom baru, yang kemudian ditampilkan. Ini menunjukkan kemampuan LLM untuk memproses data secara massal dan mengekstrak atau menghasilkan informasi spesifik dari setiap entri. Contoh yang dibahas adalah analisis sentimen, tetapi konsep ini dapat diterapkan untuk berbagai tugas seperti peringkasan, kategorisasi, atau ekstraksi entitas.
Potensi Bonus Task: Integrasi Fitur LLM Lanjutan

Jika Anda memilih untuk melanjutkan ke Bonus Task, Anda memiliki kesempatan untuk meningkatkan salah satu aplikasi Anda dengan fitur-fitur canggih seperti Multimodal Interaction, Agentic Behavior/Tool Use/Function Calling, atau Retrieval Augmented Generation (RAG). Mengimplementasikan fitur-fitur ini tidak hanya akan menunjukkan pemahaman mendalam Anda tentang kemampuan LLM, tetapi juga kemampuan Anda untuk membangun sistem AI yang lebih kompleks dan cerdas. Ini akan menambah nilai signifikan pada proyek Anda, misalnya, dengan memungkinkan chatbot Anda untuk memproses gambar, memanggil fungsi eksternal untuk mendapatkan informasi real-time, atau memberikan jawaban yang lebih akurat berdasarkan basis pengetahuan spesifik.
Kesiapan Pengumpulan

Dengan selesainya kedua tugas inti dan didokumentasikannya dengan baik di notebook Anda (termasuk kode, komentar, bagian markdown untuk deskripsi, dan contoh interaksi/output), Anda telah memenuhi persyaratan utama dari tugas gabungan Minggu 5 & 6 ini. Pastikan Anda telah mengulas semua bagian dari rubrik penilaian untuk memastikan tidak ada detail yang terlewat, terutama terkait keamanan API Key dan format pengumpulan di GitHub.

Selamat atas penyelesaian tugas yang komprehensif ini! Ini adalah langkah besar dalam memahami dan menerapkan kekuatan LLM secara programatik.
